Function Calls:

1. Create chatbots that answer questions by **calling external tools** (e.g., like ChatGPT Plugins)

2. Convert **natural language** into API calls or database queries

3. Extract **structured** data from text

### Example 1- Send Email

Email Anya to see if she wants to get coffee next Friday

`send_email(to: string, body: string)`

## Example 2 - Weather API

 What's the weather like in Bengaluru?

 `get_current_weather(location: string, unit: 'celsius' | 'fahrenheit')`


### Example 3 - Natural Language to Function/SQL

Who are my top ten customers this month?

`get_customers_by_revenue(start_date: string, end_date: string, limit: int)`

How many orders did Acme, Inc. place last month?

`sql_query(query: string)`

## Example 4 - Extract Structured Data

Extract details of the names mentioned in this Wikipedia article .....

`extract_people_data(people: [{name: string, birthday: string, location: string}])`

# Code Example

In [3]:
!pip install openai termcolor -q

In [4]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import requests

In [5]:
# This particular model has been fine tuned to know when to call functions
GPT_MODEL = "gpt-3.5-turbo-0613"

In [6]:
from apikey import OPENAIKEY
openai.api_key = OPENAIKEY

In [ ]:
# Implementation with POST request (could as well use the openai library)
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e
        

In [ ]:
# Implementation of chat as a class
class Conversation:
    def __init__(self):
        # Sequence of messages (initially delcared as an empty list)
        self.conversation_history = []

    # Add new messages to the conversation history
    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    # Display the conversation history
    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

Function Specification

In [ ]:
# Each callable function must be described as a JSON in a list of callable functions with the format below:
# Notice how there's no need to define the function imperatively, just delcaratively.
# The model will then generate a JSON representation of the function to be called, and the parameters.
# An external function will then take these parameters and be called.
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

In [ ]:
# Initiate conversation
conversation = Conversation()

In [ ]:
# Insert the first message
conversation.add_message("user", "what is the weather like today in Bangalore")

In [ ]:
# without function call, send first message to ChatGPT (notice the field "functions=None")
chat_response = chat_completion_request(
    conversation.conversation_history,
    functions = None
)
print(chat_response)

<Response [200]>


In [ ]:
chat_response.text

'{\n  "id": "chatcmpl-7RZo5FDtr0W7lGKrkHOQKrohCGrbe",\n  "object": "chat.completion",\n  "created": 1686806853,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "I\'m sorry, but as an AI language model, I don\'t have real-time data. I suggest checking a reliable weather website or using a weather application to get the current weather information for Bangalore."\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 25,\n    "completion_tokens": 40,\n    "total_tokens": 65\n  }\n}\n'

In [ ]:
#with function call (notice the field "functions=<list_of_functions>")
chat_response = chat_completion_request(
    conversation.conversation_history,
    functions = functions
)

chat_response.json()

{'id': 'chatcmpl-7RZpmBBt8fE6XktaWHx7Xz2rWsDtG',
 'object': 'chat.completion',
 'created': 1686806958,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n  "location": "Bangalore",\n  "format": "celsius"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 102, 'completion_tokens': 25, 'total_tokens': 127}}

{'id': 'chatcmpl-7RZo8UUNSRZx1rkiLqAxRsut2zlOH',
 'object': 'chat.completion',
 'created': 1686806856,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n"location": "Bangalore",\n"format": "celsius"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 102, 'completion_tokens': 23, 'total_tokens': 125}}

In [ ]:
import json
json.loads(chat_response.json()['choices'][0]['message']['function_call']['arguments'])

{'location': 'Bangalore', 'format': 'celsius'}

In [ ]:
conversation.add_message("user", "what is the weather like today in Berlin")


In [ ]:
#with function call
chat_response = chat_completion_request(
    conversation.conversation_history,
    functions = functions
)

chat_response.json()

{'id': 'chatcmpl-7RaGE9SXdoe7AfbK5ApBYmxXktjIs',
 'object': 'chat.completion',
 'created': 1686808598,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n  "location": "Berlin",\n  "format": "celsius"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 140, 'completion_tokens': 24, 'total_tokens': 164}}

In [ ]:
import json
json.loads(chat_response.json()['choices'][0]['message']['function_call']['arguments'])

{'location': 'Berlin', 'format': 'celsius'}